In [35]:
import geopandas as gpd

# Load the polygon GeoJSON
polygon_gdf = gpd.read_file("polygon.geojson")

# Load the county boundaries (replace 'counties.geojson' with your actual data file)
counties_gdf = gpd.read_file("counties2.geojson")

# Ensure the CRS (Coordinate Reference System) matches
polygon_gdf = polygon_gdf.to_crs(counties_gdf.crs)

# Calculate the total area of the farm (polygon)
farm_area = polygon_gdf.geometry.area.sum()

# Perform spatial join to find counties that intersect with the polygon
intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]

# Calculate the intersection area for each county
intersecting_counties["intersection_area"] = intersecting_counties.geometry.intersection(
    polygon_gdf.unary_union
).area

# Calculate the percentage of the farm area covered by each county
intersecting_counties["percentage_of_farm"] = (
    intersecting_counties["intersection_area"] / farm_area * 100
)

# Display results
intersecting_counties

/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  farm_area = polygon_gdf.geometry.area.sum()
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:20: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  polygon_gdf.unary_union
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to

,OBJECTID,DNR_REGION_NAME,DNR_CNTY_CODE,COUNTY_NAME,COUNTY_FIPS_CODE,SHAPEAREA,SHAPELEN,geometry,intersection_area,percentage_of_farm
11,972,South Central Region,13,Dane,25,3.204610e+09,231224.530044,"POLYGON ((-89.36309 43.28132, -89.36312 43.294...",0.000347,29.552033
16,977,South Central Region,14,Dodge,27,2.348412e+09,195437.668076,"POLYGON ((-88.40097 43.61644, -88.40114 43.630...",0.000511,43.576725
18,979,South Central Region,11,Columbia,21,2.060374e+09,222534.705663,"POLYGON ((-89.12693 43.64276, -89.12692 43.643...",0.000315,26.871242


In [37]:
def get_countyinfo(geojson):
    polygon_gdf = gpd.read_file(geojson)
    counties_gdf = gpd.read_file("counties2.geojson")
    polygon_gdf = polygon_gdf.to_crs(counties_gdf.crs)
    farm_area = polygon_gdf.geometry.area.sum()
    intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]
    intersecting_counties["intersection_area"] = intersecting_counties.geometry.intersection(
        polygon_gdf.unary_union
    ).area
    intersecting_counties["percentage_of_farm"] = (
        intersecting_counties["intersection_area"] / farm_area * 100
    )
    return intersecting_counties[["COUNTY_NAME", "percentage_of_farm"]].to_dict(orient="records")

In [ ]:
# rename county column from COUNTY_NAME to county
intersecting_counties.rename(columns={"COUNTY_NAME": "county"}, inplace=True)
intersecting_counties[["COUNTY_NAME", "percentage_of_farm"]].to_dict(orient="records")

[{'COUNTY_NAME': 'Dane', 'percentage_of_farm': 29.5520332347441},
 {'COUNTY_NAME': 'Dodge', 'percentage_of_farm': 43.57672461782967},
 {'COUNTY_NAME': 'Columbia', 'percentage_of_farm': 26.871242147426543}]

In [2]:
import pandas as pd
df = pd.read_csv("inner_join.csv")

/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_53478/2935030983.py:2: DtypeWarning: Columns (1,2,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("inner_join.csv")


In [6]:
df

,Date,High temperature,Low temperature,Precipitation,Snow,Snow depth,Heating degree days,Cooling degree days,County,EVENT_TYPE,STATE
0,2024-07-05,69,61,0.31,0.0,0,0,0,BROWN,Flooding,WISCONSIN
1,2024-07-04,82,63,0.02,0.0,0,0,8,BROWN,Normal Weather,WISCONSIN
2,2024-07-03,86,64,0.02,0.0,0,0,10,BROWN,Normal Weather,WISCONSIN
3,2024-07-02,70,58,1.31,0.0,0,1,0,BROWN,Normal Weather,WISCONSIN
4,2024-07-01,74,50,0.00,0.0,0,3,0,BROWN,Normal Weather,WISCONSIN
...,...,...,...,...,...,...,...,...,...,...,...
133406,2019-01-22,13,-7,0.00,0.0,5,62,0,VERNON,Precipitation,WISCONSIN
133407,2019-01-21,4,-9,0.00,0.0,6,67,0,VERNON,Normal Weather,WISCONSIN
133408,2019-01-20,11,-9,0.00,0.0,6,64,0,VERNON,Normal Weather,WISCONSIN
133409,2019-01-19,15,4,0.48,5.8,6,55,0,VERNON,Precipitation,WISCONSIN


In [13]:
df.sort_values("Date", ascending=True)

,Date,High temperature,Low temperature,Precipitation,Snow,Snow depth,Heating degree days,Cooling degree days,County,EVENT_TYPE,STATE
18340,2019-01-01,24,9,0.12,5.0,9,48,0,IRON,Precipitation,WISCONSIN
18339,2019-01-02,11,-10,0.00,0.0,9,64,0,IRON,Normal Weather,WISCONSIN
18338,2019-01-03,22,-10,0.00,0.0,9,59,0,IRON,Normal Weather,WISCONSIN
18337,2019-01-04,36,22,0.00,0.0,9,36,0,IRON,Normal Weather,WISCONSIN
18336,2019-01-05,46,22,0.00,0.0,9,31,0,IRON,Normal Weather,WISCONSIN
...,...,...,...,...,...,...,...,...,...,...,...
109021,2024-07-30,85,60,0.01,0.0,0,0,8,DOUGLAS,Normal Weather,WISCONSIN
52465,2024-07-30,86,67,0.00,0.0,0,0,12,POLK,Extreme Hazards,WISCONSIN
20379,2024-07-30,85,64,0.03,0.0,0,0,10,PORTAGE,Extreme Hazards,WISCONSIN
104922,2024-07-30,86,65,0.38,0.0,0,0,11,MARATHON,Extreme Hazards,WISCONSIN


In [114]:
import os
import pandas as pd

# Path to the directory containing CSV files
input_dir = '/Users/darshan/Documents/Projects/forefarm/normal'
output_file = 'processed_data.csv'

def safe_float(value):
    try:
        return float(value)
    except ValueError:
        return float(0)

# Initialize a list to store processed data
processed_data = []

# Process each CSV file in the directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_dir, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Ensure the DataFrame is sorted by date in descending order
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date', ascending=False)
        dats = []
        for i in range(7):
            dats.append(df.iloc[i:i+7])

        super_dats=[]
        for days in dats:
            # Flatten the data into a single feature vector (7 days * 7 predictors)
            feature_vec = []
            for _, row in days.iterrows():
                feature_vec.extend([
                    safe_float(row['High temperature']),
                    safe_float(row['Low temperature']),
                    safe_float(row['Precipitation']),
                    safe_float(row['Snow']),
                    safe_float(row['Snow depth']),
                    safe_float(row['Heating degree days']),
                    safe_float(row['Cooling degree days'])
                ])
            super_dats.append(feature_vec)
        
        # Add the County Name and the feature vector
        processed_data.append([file_name.rsplit('.', 1)[0], super_dats])

# Create a DataFrame for the processed data
output_df = pd.DataFrame(processed_data, columns=['county', 'features'])

# Save to a new CSV file
output_df.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")


Processed data saved to processed_data.csv


In [115]:
df = pd.read_csv("processed_data.csv")
df

,county,features
0,brown,"[[53.0, 35.0, 0.0, 0.0, 0.0, 21.0, 0.0, 49.0, ..."
1,langlade,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,lincoln,"[[52.0, 28.0, 0.0, 0.0, 0.0, 25.0, 0.0, 48.0, ..."
3,ashland,"[[51.0, 30.0, 0.0, 0.0, 0.0, 24.0, 0.0, 48.0, ..."
4,trempealeau,"[[56.0, 37.0, 0.0, 0.0, 0.0, 18.0, 0.0, 51.0, ..."
...,...,...
68,shawano,"[[51.0, 29.0, 0.0, 0.0, 0.0, 25.0, 0.0, 46.0, ..."
69,iowa,"[[52.0, 47.0, 0.01, 0.0, 0.0, 15.0, 0.0, 52.0,..."
70,washington,"[[51.0, 45.0, 0.04, 0.0, 0.0, 17.0, 0.0, 50.0,..."
71,grant,"[[47.0, 39.0, 0.0, 0.0, 0.0, 22.0, 0.0, 46.0, ..."


In [94]:
import ast
import numpy as np
df = pd.read_csv("processed_data.csv")
def get_countyvec(county):
    global df
    return df[df["county"] == county]["features"].apply(ast.literal_eval).values[0]

In [77]:
ctyinfo = {"COUNTY_NAME":"Rock","percentage_of_farm":{"$numberDouble":"0.5858492011717809"}},{"COUNTY_NAME":"Green","percentage_of_farm":{"$numberDouble":"1.9575463441906389"}},{"COUNTY_NAME":"Iowa","percentage_of_farm":{"$numberDouble":"0.34982710585646076"}},{"COUNTY_NAME":"Dane","percentage_of_farm":{"$numberDouble":"63.932919277473665"}},{"COUNTY_NAME":"Sauk","percentage_of_farm":{"$numberDouble":"0.006821967285448092"}},{"COUNTY_NAME":"Columbia","percentage_of_farm":{"$numberDouble":"32.35680643727202"}},{"COUNTY_NAME":"Marquette","percentage_of_farm":{"$numberDouble":"0.20229057327063688"}},{"COUNTY_NAME":"Green Lake","percentage_of_farm":{"$numberDouble":"0.6079390934791503"}}

In [117]:
ctyinfo = {"COUNTY_NAME":"Rock","percentage_of_farm":{"$numberDouble":"0.5858492011717809"}},{"COUNTY_NAME":"Green","percentage_of_farm":{"$numberDouble":"1.9575463441906389"}},{"COUNTY_NAME":"Iowa","percentage_of_farm":{"$numberDouble":"0.34982710585646076"}},{"COUNTY_NAME":"Dane","percentage_of_farm":{"$numberDouble":"63.932919277473665"}},{"COUNTY_NAME":"Sauk","percentage_of_farm":{"$numberDouble":"0.006821967285448092"}},{"COUNTY_NAME":"Columbia","percentage_of_farm":{"$numberDouble":"32.35680643727202"}},{"COUNTY_NAME":"Marquette","percentage_of_farm":{"$numberDouble":"0.20229057327063688"}},{"COUNTY_NAME":"Green Lake","percentage_of_farm":{"$numberDouble":"0.6079390934791503"}}

vecs = []
for cty in ctyinfo:
    info = cty["COUNTY_NAME"].lower()
    county_vecs = get_countyvec(info.replace(" ", ""))
    vecs.append(county_vecs)  # appending list of 49-value vectors (7 lists of 49 values)

# Convert to numpy array with shape (n_counties, 7, 49) where:
# - n_counties = number of counties
# - 7 = number of lists (7 vectors for each county)
# - 49 = number of values in each vector
vecs = np.array(vecs, dtype=float)

# Step 2: Extract weights and normalize them
weights = [float(cty["percentage_of_farm"]["$numberDouble"]) for cty in ctyinfo]
weights = np.array(weights, dtype=float)
weights = weights / weights.sum()  # Normalize weights

# Step 3: Compute the weighted average for each of the 7 vectors
weighted_avg_vecs = []
for i in range(7):  # For each of the 7 vectors
    weighted_avg_vec = (vecs[:, i, :] * weights[:, np.newaxis]).sum(axis=0)  # Compute the weighted average across counties
    weighted_avg_vecs.append(weighted_avg_vec)

# Convert the result into a list of weighted average vectors (7 vectors with 49 values each)
weighted_avg_vecs = np.array(weighted_avg_vecs)

# Print the resulting weighted average vectors
len(weighted_avg_vecs)

7

In [112]:
#Author: Pranav Mehrotra, Sunkar Bolat

def convert(value):
    
    try:
        
        # Convert string to float if possible
        return float(value) if isinstance(value, str) else value
        
    except ValueError:
        
        # Replace non-numeric values with 0
        return 0



def preprocessing_ml_model_2():

    import pandas as pd

    df = pd.read_csv('inner_join.csv', low_memory = False)

    df = df.drop(['Date', 'County', 'EVENT_TYPE', 'STATE'], axis=1)

    X = []
    
    Y = []

    df_as_list = df.values.tolist()
        
    for i in range(0, len(df) - 13):  # Stop at len(df) - 13 to prevent out-of-bounds.
            
        # Collect features (rows i to i+6)
            
        feature = sum(df_as_list[i:i + 7], [])  # Combine rows i to i+6 into a single list

        # Collect the label (row i+13)
        
        label = df_as_list[i + 13]

        # Append to X and Y
        
        X.append(feature)
        
        Y.append(label)

     # Step 3: Convert any non-numeric value (like null or T) in X or Y to 0, ensuring all elements are integers
    
    X = [[convert(item) for item in row] for row in X]
    
    Y = [[convert(item) for item in row] for row in Y]
                
    return X,Y


    

def trained_ml_1_model():

    import pandas as pd 

    from sklearn.ensemble import RandomForestClassifier
    
    from sklearn.metrics import accuracy_score

    from sklearn.model_selection import train_test_split
    
    from sklearn.metrics import accuracy_score,classification_report
    
    df = pd.read_csv('inner_join.csv', low_memory = False)
    
    X = df[['High temperature', 'Low temperature', 'Precipitation', 'Snow', 'Snow depth', 
        'Heating degree days', 'Cooling degree days']].values.tolist()
    
    Y = df['EVENT_TYPE'].values.tolist()

    # Step 3: Convert any non-numeric value (like null or T) in X to 0, ensuring all elements are integers
    
    for i in range(len(X)):
        
        for j in range(len(X[i])):
            
            try:
                
                if isinstance(X[i][j], str):  # Check if the element is a string
                    
                    X[i][j] = float(X[i][j])  # Replace the string value with 0
            except:
                
                X[i][j] = 0
                
                continue
                
    # Split the EVENT_TYPE list (y) and the features list (X) into training and testing sets

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Create a fitted Random Forest classifier model
    
    model = RandomForestClassifier(random_state=42, max_depth=300)
    
    model.fit(X_train, Y_train)
    
    # Predict on the test set
    
    Y_pred = model.predict(X_test)
    
    # Fit the model to the training data (X_train and y_train)

    model.fit(X_train, Y_train)

    # Predict the event types for the test set
    
    Y_pred = model.predict(X_test)

    return model, classification_report(Y_test, Y_pred)


def trained_ml_2_model():
    
    from sklearn.model_selection import train_test_split

    from sklearn.ensemble import RandomForestRegressor

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

    X,Y = preprocessing_ml_model_2()
    
    # Split the EVENT_TYPE list (y) and the features list (X) into training and testing sets

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Create the Decision Tree Regressor model

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Fit the model to the training data (X_train and y_train)

    model.fit(X_train, Y_train)
    
    # Predict the values for the test set

    Y_pred = model.predict(X_test)
    
    regression_report = {
        
        'Mean Absolute Error (MAE)': mean_absolute_error(Y_test, Y_pred),
        
        'Mean Squared Error (MSE)': root_mean_squared_error(Y_test, Y_pred),
        
        'Root Mean Squared Error (RMSE)': root_mean_squared_error(Y_test, Y_pred, squared=False),
        
        'R-squared (R²) Score': r2_score(Y_test, Y_pred),
        
        'Explained Variance Score': explained_variance_score(Y_test, Y_pred),
    
    }
    
    return model, regression_report
    

def save_model(model, file_name):

    import pickle

    with open(str(file_name) + ".pkl", 'wb') as file:
        
        pickle.dump(model, file)


def load_model(file_path):

    import pickle
    
    with open(file_path, 'rb') as file:
        
        loaded_model = pickle.load(file)
        
        return loaded_model


def predictor(feature_vector, ml_1_model, ml_2_model):

    intermediate_vector = ml_2_model.predict(feature_vector)
    
    output_label = ml_1_model.predict(intermediate_vector)
    
    return output_label

In [ ]:
m1 = load_model("ml_1_model.pkl")
m2 = load_model("ml_2_model.pkl")

array(['Normal Weather'], dtype='<U20')

In [129]:
res = []
for vec in weighted_avg_vecs:
    res.append(predictor([vec.tolist()], m1, m2).tolist()[0])

In [130]:
res

['Normal Weather',
 'Normal Weather',
 'Normal Weather',
 'Normal Weather',
 'Normal Weather',
 'Normal Weather',
 'Normal Weather']